In [55]:
import pandas as pd
import numpy as np

In [56]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

In [57]:
train_data['common_ts'] = pd.to_datetime(train_data['common_ts'], unit='ms')
test_data['common_ts'] = pd.to_datetime(test_data['common_ts'], unit='ms')

In [58]:
def udmap_onethot(d):
    v = np.zeros(9)
    if d == 'unknown':
        return v
    
    d = eval(d)
    for i in range(1, 10):
        if 'key' + str(i) in d:
            v[i-1] = d['key' + str(i)]
            
    return v

In [59]:
train_udmap_df = pd.DataFrame(np.vstack(train_data['udmap'].apply(udmap_onethot)))
test_udmap_df = pd.DataFrame(np.vstack(test_data['udmap'].apply(udmap_onethot)))

In [60]:
train_udmap_df.columns = ['key' + str(i) for i in range(1, 10)]
test_udmap_df.columns = ['key' + str(i) for i in range(1, 10)]

In [61]:
train_data = pd.concat([train_data, train_udmap_df], axis=1)
test_data = pd.concat([test_data, test_udmap_df], axis=1)

In [62]:
train_data['eid_freq'] = train_data['eid'].map(train_data['eid'].value_counts())
test_data['eid_freq'] = test_data['eid'].map(train_data['eid'].value_counts())

train_data['eid_mean'] = train_data['eid'].map(train_data.groupby('eid')['target'].mean())
test_data['eid_mean'] = test_data['eid'].map(train_data.groupby('eid')['target'].mean())

In [63]:
train_data['udmap_isunknown'] = (train_data['udmap'] == 'unknown').astype(int)
test_data['udmap_isunknown'] = (test_data['udmap'] == 'unknown').astype(int)

In [64]:
train_data['common_ts_hour'] = train_data['common_ts'].dt.hour
test_data['common_ts_hour'] = test_data['common_ts'].dt.hour

In [65]:
train_data

,uuid,eid,udmap,common_ts,x1,x2,x3,x4,x5,x6,...,key4,key5,key6,key7,key8,key9,eid_freq,eid_mean,udmap_isunknown,common_ts_hour
0,0,26,"{""key3"":""67804"",""key2"":""650""}",2023-07-18 09:44:28.244,4,0,41,107,206,1,...,0.0,0.0,0.0,0.0,0.0,0.0,174811,0.072707,0,9
1,1,26,"{""key3"":""67804"",""key2"":""484""}",2023-07-11 13:42:21.469,4,0,41,24,283,4,...,0.0,0.0,0.0,0.0,0.0,0.0,174811,0.072707,0,13
2,2,8,unknown,2023-07-15 07:49:53.040,4,0,41,71,288,4,...,0.0,0.0,0.0,0.0,0.0,0.0,51591,0.097401,1,7
3,3,11,unknown,2023-07-16 00:36:55.688,1,3,41,17,366,1,...,0.0,0.0,0.0,0.0,0.0,0.0,53698,0.098421,1,0
4,4,26,"{""key3"":""67804"",""key2"":""650""}",2023-07-16 07:15:51.442,0,3,41,92,383,4,...,0.0,0.0,0.0,0.0,0.0,0.0,174811,0.072707,0,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
620351,620351,2,"{""key4"":""14"",""key5"":""14""}",2023-07-10 12:07:30.527,4,0,41,104,37,4,...,14.0,14.0,0.0,0.0,0.0,0.0,51071,0.139414,0,12
620352,620352,8,unknown,2023-07-16 06:51:42.305,4,0,41,104,37,1,...,0.0,0.0,0.0,0.0,0.0,0.0,51591,0.097401,1,6
620353,620353,41,"{""key1"":""274"",""key2"":""446""}",2023-07-09 08:41:18.584,0,2,41,104,37,4,...,0.0,0.0,0.0,0.0,0.0,0.0,17539,0.377388,0,8
620354,620354,26,"{""key3"":""67804"",""key2"":""650""}",2023-07-18 04:02:40.171,4,0,41,17,366,1,...,0.0,0.0,0.0,0.0,0.0,0.0,174811,0.072707,0,4


In [71]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

In [67]:
X_train, X_test, y_train, y_test = train_test_split(train_data.drop(['udmap', 'common_ts', 'uuid', 'target'], axis=1),train_data['target'], test_size = 0.3, random_state = 4)

In [68]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

In [69]:
clf = LogisticRegression()
clf.fit(X_train,y_train)

c:\Users\shining3d\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [73]:
y_pred = clf.predict(X_test)
print(accuracy_score(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

0.8577484995190938
[[159614     58]
 [ 26416     19]]
              precision    recall  f1-score   support

           0       0.86      1.00      0.92    159672
           1       0.25      0.00      0.00     26435

    accuracy                           0.86    186107
   macro avg       0.55      0.50      0.46    186107
weighted avg       0.77      0.86      0.79    186107



In [52]:
import lightgbm as lgb
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier()
clf.fit(
    train_data.drop(['udmap', 'common_ts', 'uuid', 'target'], axis=1),
    train_data['target']
)

DecisionTreeClassifier()

In [54]:
pd.DataFrame({
    'uuid': test_data['uuid'],
    'target': clf.predict(test_data.drop(['udmap', 'common_ts', 'uuid'], axis=1))
}).to_csv('submit2.csv', index=None)